# Dados do IPCA Geral e Desagregações para Indústria de Transformação - Fonte Banco Central

Esse código faz o carregamento e devida guarda de dados do IPCA (Índice Nacional de Preços ao Consumidor Amplo) baixados do sistema SGS do Banco Central


https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

In [64]:
# Importando bibliotecas necessárias:

import requests # biblioteca para acessar e interagir com APIs e páginas da web
import pandas as pd
import numpy as np

In [65]:
# Biblioteca para Conversão de Formato do Texto do Nome das Colunas para a Forma Normalizada de Decomposição:

import unicodedata

In [66]:
# Bibliotecas de Visualização de Dados:

import ipywidgets as widgets
from IPython.display import display

import plotly.graph_objects as go

import plotly.express as px

In [67]:
# Carregando os dados do IPCA do Banco Central do Brasil que estão em arquivo CSV:

df_ipca_bc = pd.read_csv('Dados/dados_ipca_bc.csv', sep=';', encoding='latin1')

In [68]:
# Substituindo o separador decimal de ',' para '.' em todas as colunas, exceto a coluna 'Data'

df_ipca_bc.iloc[:, 1:] = df_ipca_bc.iloc[:, 1:].apply(lambda x: x.str.replace(',', '.'))

In [69]:
# Substituindo valores "-" por NaN para evitar problemas de conversão de tipos:

df_ipca_bc.replace("-", np.nan, inplace=True)

In [70]:
df_ipca_bc



,Data,433 - Índice nacional de preços ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Índice de Preços ao Consumidor-Amplo (IPCA) - Bens não-duráveis - Var. % mensal,10842 - Índice de Preços ao Consumidor-Amplo (IPCA) - Bens semi-duráveis - Var. % mensal,10843 - Índice de Preços ao Consumidor-Amplo (IPCA) - Duráveis - Var. % mensal,10844 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Serviços - Var. % mensal,11428 - Índice nacional de preços ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Preços ao Consumidor - Amplo (IPCA) - Alimentação no domicílio - Var. % mensal
0,01/1980,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02/1980,4.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03/1980,6.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/1980,5.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/1980,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
541,02/2025,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
542,03/2025,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
543,04/2025,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83
544,05/2025,0.26,0.70,0.08,0.25,-0.17,0.18,0.11,0.05,0.02


In [71]:
#Conversão das colunas com dados numéricos em formato string para float

cols = df_ipca_bc.columns
df_ipca_bc[cols[1:]] = df_ipca_bc[cols[1:]].apply(pd.to_numeric, errors='coerce')

In [72]:
df_ipca_bc.iloc[1,1]

4.62

In [73]:
type(df_ipca_bc.iloc[1,1])

numpy.float64

In [74]:
# Criação de uma cópia do dataframe excluindo a última linha, que não contém dados válidos:

df_ipca_bc1 = df_ipca_bc.iloc[:-1].copy()

In [75]:
df_ipca_bc1

,Data,433 - Índice nacional de preços ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Índice de Preços ao Consumidor-Amplo (IPCA) - Bens não-duráveis - Var. % mensal,10842 - Índice de Preços ao Consumidor-Amplo (IPCA) - Bens semi-duráveis - Var. % mensal,10843 - Índice de Preços ao Consumidor-Amplo (IPCA) - Duráveis - Var. % mensal,10844 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Serviços - Var. % mensal,11428 - Índice nacional de preços ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Preços ao Consumidor - Amplo (IPCA) - Alimentação no domicílio - Var. % mensal
0,01/1980,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02/1980,4.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03/1980,6.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/1980,5.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/1980,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
540,01/2025,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
541,02/2025,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
542,03/2025,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
543,04/2025,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


## Ajuste do Formato da Data

In [76]:
#Será ajustada a data para o formato AAAA - MM - DD

#Download da biblioteca de formatação de data
from datetime import datetime

In [77]:
# Criação de um dataframe do tipo datetime

date_rng = pd.date_range(start = '1/31/1980', 
                         end = '5/31/2025', 
                         freq = 'M')

In [78]:
#Checando se o comprimento do dataframe de data é igual ao comprimento do dataframe do ipp

len(date_rng) == len(df_ipca_bc1.index)

True

In [79]:
#Inclusão do dataframe date_rng no dataframe df_ipp_var

df_ipca_bc1.insert(0,        # coluna de indice 0
                  'Mes',    # nome da nova coluna
                  date_rng)  # dados

In [80]:
#Exclusão da coluna "Data"

df_ipca_bc1 = df_ipca_bc1.drop(["Data"], axis = 1)

In [81]:
df_ipca_bc1

,Mes,433 - Índice nacional de preços ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Índice de Preços ao Consumidor-Amplo (IPCA) - Bens não-duráveis - Var. % mensal,10842 - Índice de Preços ao Consumidor-Amplo (IPCA) - Bens semi-duráveis - Var. % mensal,10843 - Índice de Preços ao Consumidor-Amplo (IPCA) - Duráveis - Var. % mensal,10844 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Serviços - Var. % mensal,11428 - Índice nacional de preços ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Preços ao Consumidor - Amplo (IPCA) - Alimentação no domicílio - Var. % mensal
0,1980-01-31,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1980-02-29,4.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1980-03-31,6.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980-04-30,5.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980-05-31,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
540,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
541,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
542,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
543,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


## Remoção de acentos e cedilhas dos nomes das colunas para evitar problemas futuros no uso de redes neurais artificiais

In [82]:
# Função para remoção de acentos e cedilha:

def remover_acentos_e_cedilha(texto):
    # Primeiro, normaliza para a forma NFD (Canonical Decomposition Form)
    texto_normalizado = unicodedata.normalize('NFD', texto)

    # Codifica para ASCII e decodifica para remover os diacríticos
    texto_sem_acentos = texto_normalizado.encode('ascii', 'ignore').decode('utf-8')

    # Troca 'ç' por 'c' e 'Ç' por 'C'
    texto_final = texto_sem_acentos.replace('ç', 'c').replace('Ç', 'C')

    return texto_final

In [83]:
#Verificação de Nomes das Colunas:

print("DataFrame Original:")
print(df_ipca_bc1.head())
print("\nNomes das colunas originais:", df_ipca_bc1.columns.tolist())

DataFrame Original:
         Mes  \
0 1980-01-31   
1 1980-02-29   
2 1980-03-31   
3 1980-04-30   
4 1980-05-31   

   433 - Índice nacional de preços ao consumidor-amplo (IPCA) - Var. % mensal  \
0                                               6.62                            
1                                               4.62                            
2                                               6.04                            
3                                               5.29                            
4                                               5.70                            

   4449 - Índice Nacional de Preços ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal  \
0                                                NaN                                               
1                                                NaN                                               
2                                                NaN                                            

In [84]:
# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_ipca_bc1.columns = [remover_acentos_e_cedilha(col) for col in df_ipca_bc1.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_ipca_bc1.head())
print("\nNomes das colunas limpos:", df_ipca_bc1.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
         Mes  \
0 1980-01-31   
1 1980-02-29   
2 1980-03-31   
3 1980-04-30   
4 1980-05-31   

   433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal  \
0                                               6.62                            
1                                               4.62                            
2                                               6.04                            
3                                               5.29                            
4                                               5.70                            

   4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal  \
0                                                NaN                                               
1                                                NaN                                               
2                                                NaN                        

In [85]:
df_ipca_bc1

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,1980-01-31,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1980-02-29,4.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1980-03-31,6.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980-04-30,5.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980-05-31,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
540,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
541,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
542,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
543,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


## Definição de início das séries temporais das variações mensais das desagregações do IPCA

A base de dados das variações mensais das desagregações do IPCA, pelo Banco Central, está completa a partir de janeiro de 1992. Portanto, o início do dataframe será janeiro de 1992.

In [86]:
# Criação do dataframe do ipca iniciando em janeiro de 1992:

df_ipca_bc_92 = df_ipca_bc1[df_ipca_bc1['Mes'] >= '1992-01-01'].copy()

In [87]:
df_ipca_bc_92

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
144,1992-01-31,25.94,28.50,28.12,13.80,29.15,25.84,25.49,NaN,NaN
145,1992-02-29,24.32,29.94,25.28,17.78,26.29,22.17,23.31,NaN,NaN
146,1992-03-31,21.40,22.48,22.16,19.09,21.07,20.74,21.20,NaN,NaN
147,1992-04-30,19.93,19.35,20.43,23.43,13.20,20.14,20.04,NaN,NaN
148,1992-05-31,24.86,19.88,26.10,23.89,21.67,26.93,25.81,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
540,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
541,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
542,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
543,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


As desagregações "27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal" e "27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal" apresentam valores somente a partir de janeiro de agosto de 1999.

Portanto, deverá ser feita mais um slice do dataframe

In [104]:
# Criação do dataframe do ipca iniciando em janeiro de 1992:

df_ipca_bc_99 = df_ipca_bc1[df_ipca_bc1['Mes'] >= '1999-08-01'].copy()

In [105]:
df_ipca_bc_99


,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
235,1999-08-31,0.56,1.55,0.31,0.08,0.46,0.10,0.24,0.37,0.17
236,1999-09-30,0.31,0.55,0.36,0.43,0.45,-0.10,0.23,0.37,0.42
237,1999-10-31,1.19,0.75,1.34,0.87,4.39,0.01,1.34,1.77,2.30
238,1999-11-30,0.95,1.86,1.06,0.65,0.68,0.09,0.65,0.52,1.66
239,1999-12-31,0.60,0.67,1.04,0.72,-0.17,0.28,0.58,0.29,1.55
...,...,...,...,...,...,...,...,...,...,...
540,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
541,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
542,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
543,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


In [106]:
#Reset do índice numérico em ordem crescente

df_ipca_bc_99.reset_index(inplace = True)

In [107]:
df_ipca_bc_99

,index,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,235,1999-08-31,0.56,1.55,0.31,0.08,0.46,0.10,0.24,0.37,0.17
1,236,1999-09-30,0.31,0.55,0.36,0.43,0.45,-0.10,0.23,0.37,0.42
2,237,1999-10-31,1.19,0.75,1.34,0.87,4.39,0.01,1.34,1.77,2.30
3,238,1999-11-30,0.95,1.86,1.06,0.65,0.68,0.09,0.65,0.52,1.66
4,239,1999-12-31,0.60,0.67,1.04,0.72,-0.17,0.28,0.58,0.29,1.55
...,...,...,...,...,...,...,...,...,...,...,...
305,540,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
306,541,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
307,542,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
308,543,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


In [108]:
# Remoção da coluna de índice criada pelo reset_index

df_ipca_bc_99 = df_ipca_bc_99.drop(columns=["index"])

In [109]:
df_ipca_bc_99


,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,1999-08-31,0.56,1.55,0.31,0.08,0.46,0.10,0.24,0.37,0.17
1,1999-09-30,0.31,0.55,0.36,0.43,0.45,-0.10,0.23,0.37,0.42
2,1999-10-31,1.19,0.75,1.34,0.87,4.39,0.01,1.34,1.77,2.30
3,1999-11-30,0.95,1.86,1.06,0.65,0.68,0.09,0.65,0.52,1.66
4,1999-12-31,0.60,0.67,1.04,0.72,-0.17,0.28,0.58,0.29,1.55
...,...,...,...,...,...,...,...,...,...,...
305,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
306,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
307,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
308,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


### Geração de arquivo em formato .xlsx contendo as variações mensais das desagregações selecionadas do IPCA de agosto/99 a maio/2025

In [111]:
# Salvando dados em um arquivo Excel dentro da pasta "IPCA_Var Mensal_BC"

df_ipca_bc_99.to_excel('IPCA_Var Mensal_BC/ipca_desag_var_mensal_maio25.xlsx', index=False)

## Estatística das Variações Mensais das Desagregações Selecionadas do IPCA

In [112]:
# Estatística descritiva do dataframe df_7060_pesos

estatisticas_var_mensal_ipca = df_ipca_bc_99.drop(columns=["Mes"]).describe()

In [113]:
estatisticas_var_mensal_ipca

,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,0.505387,0.577065,0.583129,0.450581,0.220935,0.483516,0.481452,0.403774,0.606419
std,0.394897,0.879121,0.767789,0.472388,0.536750,0.373181,0.376887,0.382964,1.071799
min,-0.680000,-4.350000,-1.110000,-1.050000,-2.590000,-0.470000,-0.350000,-0.900000,-1.840000
25%,0.260000,0.180000,0.095000,0.132500,-0.077500,0.262500,0.220000,0.190000,-0.085000
50%,0.455000,0.415000,0.540000,0.440000,0.185000,0.440000,0.450000,0.370000,0.530000
75%,0.685000,0.887500,0.967500,0.750000,0.457500,0.620000,0.690000,0.580000,1.165000
max,3.020000,4.290000,4.420000,1.880000,4.390000,2.280000,2.530000,2.300000,6.680000


### Geração de arquivo em formato .xlsx contendo as estatísticas das desagregações selecionadas do IPCA de agosto/99 a maio/2025

In [114]:
# Salvando estatísticas em um arquivo Excel dentro da pasta "IPCA_Var Mensal_BC"
estatisticas_var_mensal_ipca.to_excel('IPCA_Var Mensal_BC/estatistica_var_mensal_maio25.xlsx')

## Geração de Gráficos

In [115]:
# Criação de um gráfico interativo para visualização dos dados de pesos do IPCA	

# Lista de colunas, excluindo "Mes" e "Indice Geral"
colunas = [col for col in df_ipca_bc_99.columns if col not in ["Mes"]]

# Widget dropdown para seleção da coluna
dropdown = widgets.Dropdown(
    options=colunas,
    description='Coluna:',
    value=colunas[0]
)

def plot_time_series(coluna):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_ipca_bc_99['Mes'], y=df_ipca_bc_99[coluna], mode='lines', name=coluna))
    fig.update_layout(title=f'Time Series - {coluna}', xaxis_title='Mes', yaxis_title='Variação Mensal (%)')
    fig.show()

widgets.interact(plot_time_series, coluna=dropdown)

interactive(children=(Dropdown(description='Coluna:', options=('433 - Indice nacional de precos ao consumidor-…

<function __main__.plot_time_series(coluna)>

In [116]:
# Criação de gráfico interativo com Plotly Express
fig = px.line(df_ipca_bc_99, x='Mes', y=df_ipca_bc_99.columns[1:], 
              title='Variação Mensal das Desagregações Selecionadas do IPCA (1999-2025)', 
              labels={'value': 'Variação Mensal (%)', 'variable': 'Desagregação'})

fig.show()

c:\Users\prisc\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

